In [1]:
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Categorical, Integer, Continuous
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
from sklearn_genetic.callbacks import DeltaThreshold, TimerStopping

### Import the data and split it in train and test sets

In [2]:
data = load_digits()
label_names = data["target_names"]
y = data["target"]
X = data["data"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Define the classifier to tune and the param grid

In [3]:
clf = DecisionTreeClassifier()

params_grid = {
    "min_weight_fraction_leaf": Continuous(0, 0.5),
    "criterion": Categorical(["gini", "entropy"]),
    "max_depth": Integer(2, 20),
    "max_leaf_nodes": Integer(2, 30),
}

### Create the CV strategy and optionally some callbacks

In [4]:
cv = StratifiedKFold(n_splits=3, shuffle=True)

delta_callback = DeltaThreshold(threshold=0.001, metric="fitness")
timer_callback = TimerStopping(total_seconds=60)

callbacks = [delta_callback, timer_callback]

### Define the GASearchCV options

In [5]:
evolved_estimator = GASearchCV(
    clf,
    cv=cv,
    scoring="accuracy",
    population_size=16,
    generations=30,
    crossover_probability=0.9,
    mutation_probability=0.05,
    param_grid=params_grid,
    algorithm="eaMuPlusLambda",
    n_jobs=-1,
    verbose=True)

### Fit the model and see some results

In [6]:
evolved_estimator.fit(X_train, y_train, callbacks=callbacks)
y_predict_ga = evolved_estimator.predict(X_test)
accuracy = accuracy_score(y_test, y_predict_ga)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	16    	0.379572	0.200348   	0.770574   	0.191189   
1  	29    	0.605258	0.146615   	0.735661   	0.270989   
2  	32    	0.718256	0.0139822  	0.738986   	0.695761   
3  	32    	0.725322	0.01201    	0.738986   	0.695761   
4  	31    	0.736128	0.00537084 	0.742311   	0.724855   
5  	31    	0.73883 	0.00319545 	0.742311   	0.730673   
6  	29    	0.740441	0.00451965 	0.74813    	0.729842   
7  	30    	0.740337	0.00517348 	0.74813    	0.729842   
INFO: DeltaThreshold callback met its criteria
INFO: Stopping the algorithm


In [7]:
print(evolved_estimator.best_params_)
print("accuracy score: ", "{:.2f}".format(accuracy))

{'min_weight_fraction_leaf': 0.01686201756501332, 'criterion': 'entropy', 'max_depth': 6, 'max_leaf_nodes': 24}
accuracy score:  0.78
